In [38]:
import torch
import torch.nn as nn
import networkx as nx
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx

import torch.nn.functional as F
import pandas as pd
import numpy as np

import re
pd.set_option('display.max_colwidth', None)

import math

from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

import transformers
from transformers import RobertaTokenizer, BertTokenizer, RobertaModel, BertModel, AdamW
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import get_linear_schedule_with_warmup
from torch.nn.utils.rnn import pad_sequence

import pprint
import time
import timeit

from graphModels import *

import random

In [7]:
MAX_LEN = 1024
CHUNK_LEN = 200
OVERLAP_LEN = int(CHUNK_LEN/2)

TRAIN_BATCH_SIZE = 8
EPOCH = 10
lr=1e-5

In [12]:
class Hi_Bert_Classification_Model_GCN(nn.Module):
    
    """ A Model for bert fine tuning, put an lstm on top of BERT encoding """

    def __init__(self, graph_type, num_class, device, adj_method, pooling_method='mean'):
        super(Hi_Bert_Classification_Model_GCN, self).__init__()
        self.graph_type = graph_type
        # self.bert_path = 'bert-base-uncased'
        # self.bert = transformers.BertModel.from_pretrained(self.bert_path)
#         self.bert = BertModel.from_pretrained("bert-base-multilingual-uncased")
        self.bert = XLMRobertaModel.from_pretrained("xlm-roberta-base")
        

        self.lstm_layer_number = 2
        'default 128 and 32'
        self.lstm_hidden_size = 128
        self.hidden_dim = 32
        
        # self.bert_lstm = nn.Linear(768, self.lstm_hidden_size)
        self.device = device
        self.pooling_method=pooling_method

        self.mapping = nn.Linear(768, self.lstm_hidden_size).to(device)

        'start GCN'
        if self.graph_type == 'gcn':
            self.gcn = GCN(input_dim=self.lstm_hidden_size, hidden_dim=32, output_dim=num_class).to(device)
        elif self.graph_type == 'gat':
            self.gcn = GAT(input_dim=self.lstm_hidden_size, hidden_dim=32, output_dim=num_class).to(device)
        elif self.graph_type == 'graphsage':
            self.gcn = GraphSAGE(input_dim=self.lstm_hidden_size, hidden_dim=32, output_dim=num_class).to(device)
        elif self.graph_type == 'linear':
            self.gcn = LinearFirst(input_dim=self.lstm_hidden_size, hidden_dim=32, output_dim=num_class).to(device)
        elif self.graph_type == 'rank':
            self.gcn = SimpleRank(input_dim=self.lstm_hidden_size, hidden_dim=32, output_dim=num_class).to(device)
        elif self.graph_type == 'diffpool':
            self.gcn = DiffPool(self.device,max_nodes=10,input_dim=self.lstm_hidden_size, hidden_dim=32, output_dim=num_class).to(device)
        elif self.graph_type == 'hipool':
            self.gcn = HiPool(self.device,input_dim=self.lstm_hidden_size, hidden_dim=32, output_dim=num_class).to(device)
            
        self.adj_method = adj_method


    def forward(self, ids, mask, token_type_ids):

        # import pdb;pdb.set_trace()
        'encode bert'
        bert_ids = pad_sequence(ids).permute(1, 0, 2).long().to(self.device)
        bert_mask = pad_sequence(mask).permute(1, 0, 2).long().to(self.device)
        bert_token_type_ids = pad_sequence(token_type_ids).permute(1, 0, 2).long().to(self.device)
        batch_bert = []
        for emb_pool, emb_mask, emb_token_type_ids in zip(bert_ids, bert_mask, bert_token_type_ids):
            results = self.bert(emb_pool, attention_mask=emb_mask, token_type_ids=emb_token_type_ids)
            batch_bert.append(results[1])

        sent_bert = torch.stack(batch_bert, 0)
        'GCN starts'
        sent_bert = self.mapping(sent_bert)
        node_number = sent_bert.shape[1]
        

        'random, using networkx'
        if self.adj_method == 'random':
            generated_adj = nx.dense_gnm_random_graph(node_number, node_number)
        elif self.adj_method == 'er':
            generated_adj = nx.erdos_renyi_graph(node_number, node_number)
        elif self.adj_method == 'binom':
            generated_adj = nx.binomial_graph(node_number, p=0.5)
        elif self.adj_method == 'path':
            generated_adj = nx.path_graph(node_number)
        elif self.adj_method == 'complete':
            generated_adj = nx.complete_graph(node_number)
        elif self.adj_method == 'kk':
            generated_adj = kronecker_generator(node_number)
        elif self.adj_method == 'watts':
            if node_number-1 > 0:
                generated_adj = nx.watts_strogatz_graph(node_number, k=node_number-1, p=0.5)
            else:
                generated_adj = nx.watts_strogatz_graph(node_number, k=node_number, p=0.5)
        elif self.adj_method == 'ba':
            if node_number - 1>0:
                generated_adj = nx.barabasi_albert_graph(node_number, m=node_number-1)
            else:
                generated_adj = nx.barabasi_albert_graph(node_number, m=node_number)
        elif self.adj_method == 'bigbird':

            # following are attention edges
            attention_adj = np.zeros((node_number, node_number))
            global_attention_step = 2
            attention_adj[:, :global_attention_step] = 1
            attention_adj[:global_attention_step, :] = 1
            np.fill_diagonal(attention_adj,1) # fill diagonal with 1
            half_sliding_window_size = 1
            np.fill_diagonal(attention_adj[:,half_sliding_window_size:], 1)
            np.fill_diagonal(attention_adj[half_sliding_window_size:, :], 1)
            generated_adj = nx.from_numpy_matrix(attention_adj)

        else:
            generated_adj = nx.dense_gnm_random_graph(node_number, node_number)


        nx_adj = from_networkx(generated_adj)
        adj = nx_adj['edge_index'].to(self.device)

        'combine starts'
        # generated_adj2 = nx.dense_gnm_random_graph(node_number,node_number)
        # nx_adj = from_networkx(generated_adj)
        # adj = nx_adj['edge_index'].to(self.device)
        # nx_adj2 = from_networkx(generated_adj2)
        # adj2 = nx_adj2['edge_index'].to(self.device)
        # adj = torch.cat([adj2, adj], 1)
        'combine ends'

        if self.adj_method == 'complete':
            'complete connected'
            adj = torch.ones((node_number,node_number)).to_sparse().indices().to(self.device)

        if self.graph_type.endswith('pool'):
            'diffpool only accepts dense adj'
            adj_matrix = nx.adjacency_matrix(generated_adj).todense()
            adj_matrix = torch.from_numpy(np.asarray(adj_matrix)).to(self.device)
            adj = (adj,adj_matrix)
        # if self.args.graph_type == 'hipool':

        # sent_bert shape torch.Size([batch_size, 3, 768])
        gcn_output_batch = []
        for node_feature in sent_bert:
            # import pdb;pdb.set_trace()

            gcn_output=self.gcn(node_feature, adj)

            'graph-level read out, summation'
            gcn_output = torch.sum(gcn_output,0)
            gcn_output_batch.append(gcn_output)

        # import pdb;
        # pdb.set_trace()

        gcn_output_batch = torch.stack(gcn_output_batch, 0)

        'GCN ends'

        # import pdb;
        # pdb.set_trace()
        return gcn_output_batch,generated_adj # (batch_size, class_number)

In [15]:
class Preprocess:
    
    def __init__(self, testPath):
        self.test_df = pd.read_csv(testPath, sep = '\t', header=0)
        self.test_df['review'] = self.test_df['headline'].str.cat(self.test_df['text'], sep=' ')
        
    
    def clean_text(self, sentence):
        if not isinstance(sentence, str):
            # If the input is not a string, return the original input
            return sentence
        cleaned_sentence = re.sub(r'[!\"#$%&\'()*+,-./:;<=>?@^_`{|}~]', ' ', sentence)
        cleaned_sentence = re.sub(r'\s+', ' ', cleaned_sentence).strip()
        return cleaned_sentence.lower()
        
    def get_clean(self):
        
        self.test_df['cleaned_text'] = self.test_df['review'].apply(self.clean_text)
        return self.test_df[['cleaned_text', 'label']]

In [16]:
pr = Preprocess("/scratch/smanduru/NLP/project/data/amazon_2048/amazon-books-2048-test.tsv")

test = pr.get_clean()

In [17]:
roberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

class CustomDataset(Dataset):
    
    def __init__(self, tokenizer, max_len, df, chunk_len=200, overlap_len=50, approach="all", max_size_dataset=None, min_len=249):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.overlap_len = overlap_len
        self.chunk_len = chunk_len
        self.approach = approach
        self.min_len = min_len
        self.max_size_dataset = max_size_dataset
        self.data, self.label = self.process_data(df)
        
    def process_data(self, df):
        self.num_class = len(set(df['label'].values))
        return df['cleaned_text'].values, df['label'].values
    
    def long_terms_tokenizer(self, data_tokenize, targets):
        long_terms_token = []
        input_ids_list = []
        attention_mask_list = []
        token_type_ids_list = []
        targets_list = []

        previous_input_ids = data_tokenize["input_ids"].reshape(-1)
        previous_attention_mask = data_tokenize["attention_mask"].reshape(-1)
        previous_token_type_ids = data_tokenize["token_type_ids"].reshape(-1)
        remain = data_tokenize.get("overflowing_tokens")
        targets = torch.tensor(targets, dtype=torch.int)
        
        start_token = torch.tensor([101], dtype=torch.long)
        end_token = torch.tensor([102], dtype=torch.long)

        total_token = len(previous_input_ids) -2 # remove head 101, tail 102
        stride = self.overlap_len - 2
        number_chunks = math.floor(total_token/stride)

        mask_list = torch.ones(self.chunk_len, dtype=torch.long)
        type_list = torch.zeros(self.chunk_len, dtype=torch.long)
        
        for current in range(number_chunks-1):
            input_ids = previous_input_ids[current*stride:current*stride+self.chunk_len-2]
            input_ids = torch.cat((start_token, input_ids, end_token))
            input_ids_list.append(input_ids)

            attention_mask_list.append(mask_list)
            token_type_ids_list.append(type_list)
            targets_list.append(targets)

        if len(input_ids_list) == 0:
            input_ids = torch.ones(self.chunk_len-2, dtype=torch.long)
            input_ids = torch.cat((start_token, input_ids, end_token))
            input_ids_list.append(input_ids)

            attention_mask_list.append(mask_list)
            token_type_ids_list.append(type_list)
            targets_list.append(targets)

        return({
            'ids': input_ids_list,
            'mask': attention_mask_list,
            'token_type_ids': token_type_ids_list,
            'targets': targets_list,
            'len': [torch.tensor(len(targets_list), dtype=torch.long)]
        })
    
    def __getitem__(self, idx):
        
        review = str(self.data[idx])
        targets = int(self.label[idx])
        data = self.tokenizer.encode_plus(
            review,
            max_length=self.max_len,
            pad_to_max_length=False,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
        
        long_token = self.long_terms_tokenizer(data, targets)
        return long_token
    
    def __len__(self):
        return self.label.shape[0]

In [19]:
test_dataset = CustomDataset(
    tokenizer = roberta_tokenizer,
    max_len = MAX_LEN,
    chunk_len = CHUNK_LEN,
    overlap_len = OVERLAP_LEN,
    df = test)

In [20]:
def my_collate1(batches):
    return [{key: torch.stack(value) for key, value in batch.items()} for batch in batches]

# train_loader = DataLoader(train_dataset,
#                           batch_size = TRAIN_BATCH_SIZE, 
#                           shuffle = True, 
#                           collate_fn = my_collate1)

# valid_loader = DataLoader(valid_dataset,
#                           batch_size = 32, 
#                           shuffle = False, 
#                           collate_fn = my_collate1)

# test_loader = DataLoader(test_dataset,
#                           batch_size = 32, 
#                           shuffle = False, 
#                           collate_fn = my_collate1)


# Define the size of the subset you want to sample for each epoch
subset_size = 3200  # Adjust this based on your available memory and training needs
subset_size_test = 500

# Custom function to randomly sample a subset
def get_subset_sampler(subset_size):
    return SubsetRandomSampler(torch.randperm(subset_size))

# Creating a custom data loader with the subset sampler
def get_data_loader(dataset, subset_size, batch_size):
    subset_sampler = get_subset_sampler(subset_size)

    # Creating PT data samplers and loaders
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=subset_sampler,
        collate_fn=my_collate1
    )

    return data_loader

# Example usage

test_loader = get_data_loader(test_dataset, subset_size_test, TRAIN_BATCH_SIZE)

In [32]:
def loss_fun(outputs, targets):
    loss = nn.CrossEntropyLoss()
    return loss(outputs, targets)

def eval_loop_fun1(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["ids"] for data in batch]  # size of 8
        mask = [data["mask"] for data in batch]
        token_type_ids = [data["token_type_ids"] for data in batch]
        targets = [data["targets"] for data in batch]  # length: 8

        with torch.no_grad():
            target_labels = torch.stack([x[0] for x in targets]).long().to(device)
            outputs, _ = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            loss = loss_fun(outputs, target_labels)
            losses.append(loss.item())

        fin_targets.append(target_labels.cpu().detach().numpy())
        fin_outputs.append(torch.softmax(outputs, dim=1).cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

def evaluate(target, predicted):
    true_label_mask = [1 if (np.argmax(x)-target[i]) ==
                       0 else 0 for i, x in enumerate(predicted)]
    nb_prediction = len(true_label_mask)
    true_prediction = sum(true_label_mask)
    false_prediction = nb_prediction-true_prediction
    accuracy = true_prediction/nb_prediction
    return{
        "accuracy": accuracy,
        "nb exemple": len(target),
        "true_prediction": true_prediction,
        "false_prediction": false_prediction,
    }

In [27]:
load_path = '/scratch/smanduru/NLP/project/saved_models/MultiLingual/' + 'best_model_hipool_xlmroberta.pth'

# Load the entire model
model = Hi_Bert_Classification_Model_GCN(graph_type = 'hipool',
                                       num_class=test_dataset.num_class,
                                       device=device,
                                       adj_method='bigbird')

model.load_state_dict(torch.load(load_path), strict=False)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda


Hi_Bert_Classification_Model_GCN(
  (bert): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0): XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [29]:
torch.cuda.empty_cache()

In [30]:
model.eval()

Hi_Bert_Classification_Model_GCN(
  (bert): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0): XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [33]:
# Evaluate the test set
predicted_probs, true_labels, losses = eval_loop_fun1(test_loader, model, device)

# Evaluate accuracy
evaluation_result = evaluate(true_labels, predicted_probs)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/tmp/ipykernel_2272995/2335672165.py:120: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matrix = nx.adjacency_matrix(generated_adj).todense()


In [34]:
evaluation_result

{'accuracy': 0.532,
 'nb exemple': 500,
 'true_prediction': 266,
 'false_prediction': 234}

In [35]:
misclassified_examples = []
classified_examples = []
for i, (true_label, predicted_prob) in enumerate(zip(true_labels, predicted_probs)):
    predicted_label = np.argmax(predicted_prob)
    if predicted_label != true_label:
        misclassified_examples.append({
            'Example Index': i,
            'True Label': true_label,
            'Predicted Label': predicted_label,
            'Predicted Probabilities': predicted_prob.tolist(),
            'Raw Input': test.iloc[i]['cleaned_text']
        })
    else:
        classified_examples.append({
            'Example Index': i,
            'True Label': true_label,
            'Predicted Label': predicted_label,
            'Predicted Probabilities': predicted_prob.tolist(),
            'Raw Input': test.iloc[i]['cleaned_text']
        })

In [36]:
# Shuffle misclassified examples
np.random.shuffle(misclassified_examples)

# Print 5 randomly selected misclassified examples
for example in misclassified_examples[:5]:
    print(f"Example Index: {example['Example Index']}")
    print(f"True Label: {example['True Label']}, Predicted Label: {example['Predicted Label']}")
    print(f"Raw Input Sentence: {example['Raw Input']}")
    print("\n" + "="*50 + "\n")
    break

Example Index: 205
True Label: 0, Predicted Label: 3
Raw Input Sentence: a thorough examination i was offered the opportunity to review the glassfish administration book by xuekun kou from packt publishing the book focuses on glassfish version but does have some coverage of version v the book is a smaller digest of a number of books published by oracle sun and the glassfish document team the book breaks down into ten chapters on version and an eleventh chapter on version i am not sure where i stand on this book i really don t like references to certain features in the table of contents which would lead you to believe it has some depth on the subject later you discover that it refers you to the actual \\ official\\ publications i noted above i think that if you give it a sub chapter designation it must actually have some depth i really found the book to cover most of the subjects in a shallow manner my other pet peeve is when code is included or not included in a book which does not wor

# Code Mixing

In [53]:
selected_examples = random.sample(classified_examples, min(10, len(classified_examples)))

In [54]:
robust_df = pd.DataFrame(selected_examples)

columns_to_drop = ['Predicted Label', 'Predicted Probabilities']
robust_df = robust_df.drop(columns=columns_to_drop)

In [55]:
robust_df

Example Index  True Label  \
0            208           4   
1            328           0   
2            443           4   
3            145           3   
4            299           0   
5            330           0   
6            242           4   
7            231           4   
8            446           0   
9            262           0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [41]:
!pip install googletrans==3.1.0a0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [56]:
from googletrans import Translator

In [57]:
def random_translate(sentence, target_language='es'):
    translator = Translator()

    # Tokenize the sentence into words
    words = sentence.split()
    # print(words)

    # Randomly choose a percentage of words to translate
    num_words_to_translate = int(len(words) * 0.2)  # Change 0.2 to your desired percentage

    # Randomly select words to translate
    words_to_translate = random.sample(words, num_words_to_translate)

    # Translate selected words to the target language
    for i, word in enumerate(words):
        if word in words_to_translate:
            try:
                translated_word = translator.translate(word, src = 'en', dest='te').text
                words[i] = translated_word
            except Exception as e:
                # Handle translation errors
                print(f"Error translating word '{word}': {e}")

    # Join the words back into a sentence
    translated_sentence = ' '.join(words)

    return translated_sentence

In [58]:
robust_df['code_mixed_sentence'] = robust_df['Raw Input'].apply(random_translate)

['a', 'wonderful', 'read', 'meet', 't', 'j', 'also', 'known', 'as', 'timothy', 'also', 'known', 'as', 'terry', 'jerry', 'and', '\\\\', 'it', 's', 'not', 'unusual\\\\', 'for', 'him', 'to', 'be', 'referred', 'to', 'as', 'tom', 'jones', 'cracking', 'myself', 'up', 'this', 'week', 'but', 'he', 'prefers', 't', 'j', 'for', 'most', 'of', 'the', 'story', 'and', 'he', 's', 'an', 'eleven', 'year', 'old', 'kid', 'with', 'an', 'adult', 's', 'problems', 'because', 'there', 'is', 'no', 'adult', 'to', 'handle', 'his', 'problems', 'for', 'him', 'when', 'we', 'first', 'encounter', 't', 'j', 'he', 's', 'sitting', 'in', 'a', 'hospital', 'waiting', 'room', 'which', 'is', 'an', 'appropriate', 'place', 'for', 'him', 'to', 'be', 'given', 'the', 'title', 'of', 'the', 'book', 't', 'j', 'is', 'waiting', 'to', 'find', 'out', 'whether', 'or', 'not', 'his', 'little', 'sister', 'angela', 'is', 'going', 'to', 'die', 'or', 'not', 'it', 's', 'a', 'good', 'hook', 'one', 'we', 've', 'seen', 'before', 'and', 'one', 'we',

['a', 'thoroughly', 'unbiblical', 'basis', 'there', 'are', 'many', 'statements', 'in', 'this', 'book', 'with', 'which', 'it', 'is', 'difficult', 'to', 'find', 'fault', '\\\\', 'the', 'bible', 'is', 'the', 'divinely', 'inspired', 'authority', 'in', 'matters', 'of', 'faith', 'doctrine', 'and', 'conduct', 'and', 'the', 'basis', 'for', 'teaching', '\\\\', '[]', 'deliverance', 'from', 'demons', 'was', 'an', 'important', 'part', 'of', 'the', 'ministry', 'of', 'jesus', 'e', 'g', 'acts', 'and', 'as', 'his', 'followers', 'it', 'is', 'also', 'part', 'of', 'ours', 'however', 'there', 'is', 'also', 'much', 'in', 'this', 'book', 'that', 'should', 'be', 'of', 'great', 'concern', 'to', 'bible', 'believing', 'christians', 'far', 'from', 'being', 'a', 'biblical', 'basis', 'peter', 'horrobin', 's', '[henceforth', 'ph]', 'book', 'is', 'really', 'a', 'polemic', 'against', 'those', 'who', 'oppose', 'what', 'i', 'shall', 'refer', 'to', 'as', 'ellel', 'grange', 'theology', '[henceforth', 'egt', ']', 'egt', '

['an', 'earnest', 'and', 'thougtful', 'effort', 'but', 'falls', 'short', 'kenneth', 'miller', 'is', 'a', 'molecular', 'biologist', 'who', 'teaches', 'at', 'brown', 'university', 'he', 'is', 'a', 'committed', 'evolutionist', 'and', 'a', 'committed', 'christian', 'his', 'goal', 'in', 'this', 'book', 'is', 'to', 'demonstrate', 'that', 'a', 'firm', 'acceptance', 'of', 'the', 'theory', 'of', 'evolution', 'is', 'not', 'incompatible', 'with', 'a', 'belief', 'in', 'the', 'judeo', 'christian', 'god', 'despite', 'a', 'very', 'sincere', 'obviously', 'well', 'thought', 'out', 'effort', 'an', 'articulate', 'presentation', 'that', 'all', 'readers', 'can', 'appreciate', 'and', 'a', 'superb', 'detailed', 'explanation', 'of', 'the', 'principles', 'of', 'darwinian', 'evolution', 'i', 'believe', 'he', 'only', 'halfway', 'succeeds', 'in', 'his', 'principle', 'aim', 'the', 'first', 'portion', 'of', 'the', 'book', 'is', 'a', 'full', 'frontal', 'assault', 'on', 'the', 'evolution', 'deniers', 'he', 'takes', '

['a', 'hard', 'hitting', 'wake', 'up', 'call', 'of', 'a', 'true', 'compilation', 'from', 'the', 'ones', 'who', 'lived', 'it', 'substance', 'abuse', 'and', 'addictions', 'are', 'killing', 'our', 'children', 'and', 'your', 'loved', 'ones', 'in', 'droves', 'on', 'a', 'daily', 'basis', 'and', 'the', 'world', 'is', 'still', 'in', 'denial', 'if', 'this', 'book', 'does', 'not', 'get', 'you', 'right', 'between', 'the', 'eyes', 'break', 'your', 'heart', 'into', 'a', 'million', 'pieces', 'cause', 'you', 'to', 'get', 'help', 'immediately', 'if', 'not', 'sooner', 'want', 'to', 'put', 'this', 'book', 'in', 'the', 'hands', 'of', 'everyone', 'you', 'know', 'make', 'it', 'required', 'reading', 'in', 'all', 'schools', 'universities', 'and', 'seek', 'it', 'out', 'as', 'complete', 'truth', 'you', 'are', 'most', 'definitely', 'in', 'that', 'category', 'and', 'don', 't', 'have', 'a', 'clue', 'what', 'your', 'loved', 'ones', 'are', 'up', 'to', 'and', 'better', 'yet', 'make', 'advanced', 'plans', 'for', 'the

['the', 'elephant', 'and', 'the', 'blind', 'men', 'a', 'mixed', 'metaphor', 'at', 'best', '\\\\', 'the', 'study', 'of', 'comparative', 'religions', 'is', 'the', 'best', 'way', 'to', 'become', 'comparatively', 'religious', '\\\\', 'ronald', 'knoxthere', 'is', 'an', 'old', 'hindu', 'folk', 'tale', 'about', 'blind', 'men', 'and', 'an', 'elephant', 'popularized', 'in', 'verse', 'by', 'john', 'godfrey', 'saxe', 'it', 'is', 'a', 'parable', 'about', 'man', 's', 'understanding', 'of', 'and', 'relationship', 'with', 'the', 'transcendent', 'six', 'blind', 'men', 'went', 'to', 'see', 'an', 'elephant', 'each', 'touched', 'the', 'elephant', 'in', 'a', 'different', 'place', 'one', 'the', 'trunk', 'one', 'his', 'tusks', 'one', 'his', 'side', 'one', 'his', 'back', 'and', 'so', 'on', 'since', 'each', 'sensed', 'the', 'elephant', 'in', 'a', 'different', 'way', 'they', 'argued', 'amongst', 'one', 'another', 'about', 'who', 'described', 'the', 'elephant', 'correctly', 'yet', 'none', 'was', 'right', 'since

['very', 'good', 'but', 'no', 'masterpiece', 'moby', 'dick', 'may', 'be', 'american', 'literature', 's', 'most', 'famous', 'work', 'only', 'the', 'adventures', 'of', 'huckleberry', 'finn', 'even', 'rivals', 'it', 'it', 'is', 'the', 'kind', 'of', 'book', 'with', 'such', 'a', 'reputation', 'that', 'it', 'is', 'impossible', 'to', 'read', 'without', 'a', 'host', 'of', 'preconceived', 'ideas', 'one', 'need', 'not', 'even', 'care', 'about', 'it', 'much', 'less', 'have', 'read', 'it', 'to', 'know', 'the', 'outline', 'and', 'perhaps', 'much', 'more', 'many', 'judge', 'it', 'before', 'reading', 'if', 'they', 'ever', 'read', 'it', 'based', 'on', 'what', 'they', 'know', 'or', 'think', 'they', 'know', 'this', 'inevitably', 'carries', 'over', 'to', 'reading', 'often', 'making', 'a', 'self', 'fulfilling', 'prophecy', 'though', 'perhaps', 'inevitable', 'this', 'is', 'unfortunate', 'as', 'it', 'prevents', 'nearly', 'everyone', 'from', 'reading', 'critically', 'and', 'seeing', 'it', 'on', 'its', 'own',

['did', 'barack', 'obama', 'senior', 'assassinate', 'tom', 'mboya', 'i', 'purchased', 'sally', 'jacobs', '\\\\', 'the', 'other', 'barack\\\\', 'in', 'hopes', 'that', 'this', 'book', 'would', 'help', 'answer', 'many', 'of', 'the', 'paternal', 'questions', 'surrounding', 'barry', 'obama', 'however', 'my', 'hopes', 'were', 'quickly', 'shattered', 'when', 'in', 'the', 'preface', 'and', 'on', 'photo', 'sally', 'states', 'with', 'no', 'explanation', 'that', 'barry', 'only', 'met', 'his', 'father', 'once', 'in', 'his', 'life', '[in', ']', 'obama', 'says', 'his', 'father', 'left', 'when', 'he', 'was', 'two', 'years', 'old', 'and', 'dreams', 'from', 'my', 'father', 'says', 'obama', 'sr', 'left', 'hawaii', 'in', 'sally', 's', 'confusing', 'story', 'telling', 'began', 'in', 'the', 'preface', 'when', 'she', 'claims', 'a', 'village', 'of', 'obama', 's', 'relatives', 'received', 'a', 'flurry', 'of', '\\\\', 'chilling', 'emails\\\\', 'before', 'her', 'arrival', 'with', 'news', 'traveling', 'fast', 'v

['i', 'got', 'my', 'health', 'back', 'and', 'so', 'can', 'you', '[[videoid', 'mouedeetmegj]]i', 'am', 'now', 'years', 'old', 'and', 'have', 'always', 'been', 'a', 'very', 'healthy', 'individual', 'and', 'trained', 'some', 'kind', 'of', 'sport', 'almost', 'every', 'day', 'since', 'i', 'can', 'remember', 'myself', 'then', 'one', 'day', 'without', 'a', 'warning', 'i', 'collapsed', 'down', 'to', 'the', 'floor', 'and', 'experienced', 'the', 'most', 'pain', 'i', 'have', 'ever', 'felt', 'i', 'had', 'no', 'idea', 'of', 'what', 'was', 'going', 'on', 'inside', 'my', 'body', 'and', 'i', 'lay', 'on', 'the', 'floor', 'for', 'probably', 'minutes', 'before', 'i', 'made', 'an', 'attempt', 'to', 'stand', 'up', 'from', 'this', 'moment', 'on', 'until', 'i', 'attended', 'the', 'september', 'clinic', 'in', 'santa', 'rosa', 'san', 'fransisco', 'i', 'was', 'in', 'constant', 'pain', 'my', 'pain', 'level', 'was', 'at', 'all', 'times', 'no', 'exceptions', 'during', 'the', 'first', 'months', 'i', 'began', 'to', 

['a', 'very', 'easy', 'to', 'read', 'personal', 'finance', 'book', 'with', 'strong', 'advice', 'too', 'i', 'picked', 'up', 'beyond', 'paycheck', 'to', 'paycheck', 'because', 'of', 'one', 'word', 'conversation', 'for', 'the', 'most', 'part', 'beyond', 'paycheck', 'to', 'paycheck', 'by', 'michael', 'b', 'rubin', 'covers', 'the', 'nuts', 'and', 'bolts', 'of', 'personal', 'finance', 'pretty', 'well', 'what', 'sets', 'it', 'apart', 'is', 'the', 'method', 'it', 'uses', 'to', 'relate', 'this', 'information', 'most', 'of', 'the', 'chapters', 'take', 'the', 'form', 'of', 'a', 'back', 'and', 'forth', 'dialogue', 'between', 'you', 'the', 'reader', 'and', 'the', 'writer', 'don', 't', 'get', 'what', 'i', 'mean', 'here', 's', 'a', 'snippet', 'from', 'the', 'first', 'chapter', '\\\\', 'you', 'why', 'is', 'my', 'money', 'worth', 'less', 'in', 'the', 'future', 'than', 'it', 'is', 'worth', 'today', 'one', 'reason', 'is', 'inflation', 'inflation', 'is', 'the', 'overall', 'trend', 'of', 'rising', 'prices'

In [59]:
robust_df['code_mixed_sentence']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [62]:
robust_df = robust_df.rename(columns={'True Label': 'label'})
robust_df = robust_df.rename(columns={'code_mixed_sentence': 'cleaned_text'})

robust_dataset = CustomDataset(
    tokenizer = roberta_tokenizer,
    max_len = MAX_LEN,
    chunk_len = CHUNK_LEN,
    overlap_len = OVERLAP_LEN,
    df = robust_df[['cleaned_text', 'label']])


robust_loader = DataLoader(robust_dataset,
                          batch_size = 2, 
                          shuffle = False, 
                          collate_fn = my_collate1)

# Evaluate the test set
predicted_probs, true_labels, losses = eval_loop_fun1(robust_loader, model, device)

# Evaluate accuracy
evaluation_result = evaluate(true_labels, predicted_probs)

/tmp/ipykernel_2272995/2335672165.py:120: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matrix = nx.adjacency_matrix(generated_adj).todense()


In [63]:
evaluation_result

{'accuracy': 0.4,
 'nb exemple': 10,
 'true_prediction': 4,
 'false_prediction': 6}

In [64]:
future తరాలు shakes head పాపం ఇది కథ isn t గురించి అమ్మ లేదా i ఉంటుంది కలిగి ఉంటాయి stopped reading అది after a కొన్ని chapters i కలిగి ఉంటాయి ఒక పాతది స్నేహితుడు whose ఫోన్ calls i dread because ఆమె కలిగి ఉంది ఉంది పై a మార్గం కు నాశనం ఆమె life కోసం a while మరియు every సమయం

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ future తరాలు shakes head పాపం ఇది కథ isn t గురించి అమ్మ లేదా i ఉంటుంది కలిగి ఉంటాయి stopped reading అది after  │
│        ▲                                                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax